In [90]:
#TODO POSSIBLE : Make A Zipcode Dictionary that refers to their code

In [91]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import pickle

In [92]:
from config import db_password

In [93]:
# Name the SQL database your are accessing
database = "RealLeads"

In [94]:
# Make your local connection to the database in PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database=database,
    user="postgres",
    password=db_password)

In [95]:
# Table selection Function
def table_select(table_name):
    table =f"select * from \"{table_name}\""
    dataFrame = pd.read_sql(table, conn)
    return dataFrame

In [96]:
# Names of the tables to be imported
id_table_df = table_select("id_table")
broker_df = table_select("broker_data_clean")
lat_lng_df = table_select("lat_lng_clean")
mortgage_df = table_select("mortgage_data_clean")
property_char_df = table_select("prop_charac_clean")
#public_rec_df = table_select("pub_rec_clean")
sales_data_df = table_select("sales_data_clean")

In [97]:
public_rec_df = pd.read_csv("../Resources/Clean_Data_12_8_22/Pub_Rec_clean.csv")


In [98]:
ml_Temp = pd.read_csv("../Resources/ml_Data_Cleaned_Supervised_Learning.csv")

In [99]:
ml_df = pd.read_csv("../Resources/ml_DataFrame_No_NAN.csv")

In [107]:
ml_Temp.head()

,Orig_List_Price,Zip_Code,Subdivision,School_District,HOA_YN,Structure_Type,Acres,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,NumberofStories,Central_Air_YN,Water,Basement_YN,Garage_YN,ExteriorMaterial,YearBuilt
0,330000.0,13,461,4,2,1,0.26,2099.0,2099.0,0.0,4.0,2.0,27,2,12,2,2,205,87
1,215500.0,13,399,4,1,3,0.06,1927.0,1575.0,352.0,3.0,2.0,27,2,12,2,1,43,105
2,200000.0,13,618,4,1,3,0.05,1150.0,1150.0,0.0,2.0,2.0,27,2,12,2,1,291,114
3,219900.0,13,190,4,2,3,0.05,1425.0,1425.0,0.0,2.0,3.0,27,2,12,2,2,270,114
4,195000.0,13,546,4,1,1,0.14,0.0,0.0,0.0,4.0,3.0,12,1,12,2,2,237,83


In [108]:
ml_df.head()

,MLSNumber,Orig_List_Price,Zip_Code,Subdivision,School_District,HOA_YN,Structure_Type,Acres,InteriorSqFt,AboveGradeSqFt,BelowGradeSqFt,Bedrooms,Baths,NumberofStories,Central_Air_YN,Water,Basement_YN,Garage_YN,ExteriorMaterial,YearBuilt
0,DENC518086,330000.0,19808,SHERWOOD PARK II,Red Clay Consolidated,Yes,Detached,0.26,2099.0,2099.0,0.0,4.0,2.0,"Main,Upper1",Yes,Public,Yes,Yes,"BrickFront,VinylSiding",1958
1,DENC518982,215500.0,19808,PEPPER RIDGE,Red Clay Consolidated,No,Interior Row/Townhouse,0.06,1927.0,1575.0,352.0,3.0,2.0,"Main,Upper1",Yes,Public,Yes,No,"AluminumSiding,Brick,VinylSiding",1976
2,DENC512992,200000.0,19808,WOODMILL,Red Clay Consolidated,No,Interior Row/Townhouse,0.05,1150.0,1150.0,0.0,2.0,2.0,"Main,Upper1",Yes,Public,Yes,No,VinylSiding,1985
3,DENC512104,219900.0,19808,FAIRWAY FALLS,Red Clay Consolidated,Yes,Interior Row/Townhouse,0.05,1425.0,1425.0,0.0,2.0,3.0,"Main,Upper1",Yes,Public,Yes,Yes,Other,1985
4,DENC506356,195000.0,19808,WASHINGTON HEIGHTS,Red Clay Consolidated,No,Detached,0.14,0.0,0.0,0.0,4.0,3.0,"Lower1,Main,Upper1",No,Public,Yes,Yes,"Concrete,Stucco",1954


In [102]:
ml_df.columns

Index(['MLSNumber', 'Orig_List_Price', 'Zip_Code', 'Subdivision',
       'School_District', 'HOA_YN', 'Structure_Type', 'Acres', 'InteriorSqFt',
       'AboveGradeSqFt', 'BelowGradeSqFt', 'Bedrooms', 'Baths',
       'NumberofStories', 'Central_Air_YN', 'Water', 'Basement_YN',
       'Garage_YN', 'ExteriorMaterial', 'YearBuilt'],
      dtype='object')

In [103]:
ml_Temp.columns

Index(['Orig_List_Price', 'Zip_Code', 'Subdivision', 'School_District',
       'HOA_YN', 'Structure_Type', 'Acres', 'InteriorSqFt', 'AboveGradeSqFt',
       'BelowGradeSqFt', 'Bedrooms', 'Baths', 'NumberofStories',
       'Central_Air_YN', 'Water', 'Basement_YN', 'Garage_YN',
       'ExteriorMaterial', 'YearBuilt'],
      dtype='object')

In [106]:
zipcode_dict = dict(zip(ml_df.Zip_Code, ml_Temp.Zip_Code))
zipcode_dict.items()

dict_items([(19808, 13), (19807, 12), (19806, 11), (19805, 10), (19804, 9), (19803, 8), (19802, 7), (19801, 6), (19720, 3), (19736, 5), (19732, 4), (19711, 2), (19707, 1)])

In [105]:
#Test Loading a machine learning model

In [111]:
filename = 'RandomForestRegressor_model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(ml_Temp.drop(columns="Orig_List_Price"), ml_Temp["Orig_List_Price"])
res = loaded_model.predict(ml_Temp.drop(columns="Orig_List_Price"))

In [112]:
res

array([330013.99      , 224268.25      , 207087.        , ...,
       514999.99      , 580620.98      , 350704.33333333])